#Imports

In [0]:
%pip install --upgrade openaI numpy==1.23.5 databricks-vectorsearch

In [0]:
dbutils.library.restartPython()

In [0]:
import os
import re
import json
import pandas as pd
from datetime import datetime, timedelta 
from openai import AssistantEventHandler, AzureOpenAI
import openai
import requests
from typing_extensions import override
import unicodedata
from databricks.vector_search.client import VectorSearchClient
from datetime import datetime

In [0]:
%run ../common/nb_init

#Functions

In [0]:
def get_date():
    current_date = datetime.now()
    formatted_date = current_date.strftime('%d-%m-%Y')
    return formatted_date

In [0]:
date = get_date()

#Connect to Client

In [0]:
secret= get_secret("openai-assistant-key")
endpoint = get_secret("openai-assistant-endpoint")
ASSISTANT_ID = get_secret("openai-assistant-id")
#ASSISTANT_Testing_ID = get_secret("openai-assistant-testing-id")

In [0]:
client = AzureOpenAI(
    api_key=secret,
    api_version="2024-05-01-preview",
    azure_endpoint=endpoint
)

#Create Assistant Tools

#Databricks VectorSearch

In [0]:
vsc = VectorSearchClient(disable_notice=True)
index = vsc.get_index(endpoint_name="uk-search-endpoint" , index_name="datif_pz_uk_dev.04_ai.search_vector_index")

In [0]:
def semantic_search(user_query):
    results = index.similarity_search(query_text=user_query, num_results=10, query_type="hybrid", score_threshold=0.5,
    columns=[
        "ID",
        "Date",
        "Channel",
        "Chunk",
        "URL",
    ],
    )
    return json.dumps(results)

In [0]:
semantic_search_tool = {
    "type": "function",
    "function": {
        "name": "semantic_search",
        "description": "Performs a semantic search.",
        "parameters": {
            "type": "object",
            "properties": {
                "user_query": {
                    "type": "string",
                    "description": "The optimized search query."
                }
            },
            "required": ["user_query"]
        }
    }
}

#Retrieve Client Files

In [0]:
files = client.files.list().data
files_sorted = sorted(files, key=lambda f: f.created_at, reverse=True)

eco_journal_file_id = None
socials_file_id = None
reputationsindex_file_id = None

for f in files_sorted[:3]:
    if f.filename == "eco_journal.csv":
        eco_journal_file_id = f.id
    elif f.filename == "socials.csv":
        socials_file_id = f.id
    elif f.filename == "reputationsindex.csv":
        reputationsindex_file_id = f.id

#Create/Update Assistant

In [0]:
instructions = f"""
Du bist ein datenintelligenter KI-Chatbot mit Zugriff auf zwei vollständig verarbeitete CSV-Dateien – eine mit Social-Media-Posts, eine mit Artikel-Interaktionen. Beide Dateien liegen in strukturierter Form vor und sind direkt analysierbar (kein Excel-Format).

📅 **Aktuelles Datum:** {date}  
Die Daten reichen historisch bis zum 01.01.2025 zurück und werden **täglich aktualisiert**.
---
📂 **Dateizugriff**

- **Verwende ausschließlich die folgenden `file_id`s für den Zugriff** – **nicht die Dateinamen**:
  - Social-Media-Daten → `file_id: {socials_file_id}`
  - Eco Journal Daten → `file_id: {eco_journal_file_id}`
  - Reputationsindex → `file_id: {reputationsindex_file_id}`
- Diese `file_id`s sind **immer aktuell**, da sie dynamisch im Systemprompt gesetzt werden.
- **Falls eine vom Nutzer genannte ID veraltet oder ungültig ist** (z. B. durch eine Datenaktualisierung), **ignoriere sie und verwende stattdessen immer die hier im Prompt angegebenen `file_id`s.**
- **Wichtig:** Nenne in deiner Antwort niemals eine ungültige oder veraltete ID.
---
🔧 **Verfügbare Tools**

1. 🧮 **Code Interpreter**  
   Für analytische Fragen wie:
   - „Welche Woche hatte die höchste Impression-Zahl?“  
   - „Welche Plattform zeigt im Schnitt die höchste Engagement-Rate?“

2. 🔍 **Semantic Search (vektorbasiert)**
   Für inhaltliche bzw. thematische Fragen wie:
   - „Gab es Beiträge über das Thema Photovoltaik?“  
   - „Wurde in einem Post Dirk Güsewell erwähnt?“

---
📂 **Datenstruktur**

`socials.csv` (file_id: {socials_file_id})

| Spalte                  | Typ     | Beschreibung                                                             |
|-------------------------|----------|--------------------------------------------------------------------------|
| ID                      | string   | Eindeutige Beitragskennung                                               |
| Date                    | string   | Veröffentlichungsdatum des Posts (Format: „YYYY-MM-DD“)                 |
| Channel                 | string   | Plattform (z. B. Facebook, Instagram)                                    |
| URL                     | string   | Direktlink zum Social-Media-Post                                         |
| PostMessage             | string   | Textinhalt des Beitrags                                                  |
| Impressions             | string   | Anzahl der Sichtungen                                                    |
| EngagementRateInPercent | string   | Engagement-Rate in Prozent (z. B. „50.0“)                                |
| PostType                | string   | Beitragsart (z. B. „Video“, „Image“)                                     |
| StrategischesThema      | string   | Strategische Zuordnung (ggf. mehrere Themen durch `&` getrennt)          |
| Themenbereich           | string   | Inhaltlicher Themencluster (ggf. mehrere Bereiche durch `&` getrennt)    |

 `eco_journal.csv` (file_id: {eco_journal_file_id})

| Spalte                   | Typ     | Beschreibung                                                             |
|--------------------------|----------|--------------------------------------------------------------------------|
| Created_Date             | string   | Veröffentlichungsdatum des Artikels (Format: „YYYY-MM-DD“)              |
| URL                      | string   | Direktlink zum Artikel (.html)                                           |
| Engaged_sessions         | string   | Anzahl der Sitzungen mit aktiver Nutzerinteraktion                      |
| Sessions                 | string   | Gesamtanzahl der Sitzungen                                              |
| Views                    | string   | Seitenaufrufe                                                            |
| User_engagement          | string   | Interaktionsdauer in Sekunden                                           |
| Total_session_duration   | string   | Gesamtdauer aller Sitzungen in Sekunden                                 |
| Bounce_rate              | string   | Absprungrate in Prozent (z. B. 9.03)                                     |
| Average_session_duration | string   | Durchschnittliche Sitzungsdauer in Sekunden                             |
| Views_per_session        | string   | Seitenaufrufe pro Sitzung                                               |
| Active_Users_per_View    | string   | Verhältnis aktiver Nutzer pro Aufruf                                    |
| Samples_read_rate        | string   | Anteil gelesener Inhaltsproben (kann `null` sein)                       |
| Active_Users             | string   | Aktive Nutzer                                                            |
| Total_Users              | string   | Gesamtzahl aller Nutzer                                                 |
| Strategische_Themen      | string   | Strategische Zuordnung (mehrere durch `&` getrennt möglich)             |
| Abstract                 | string   | Redaktionelle Zusammenfassung                                           |
| Themenbereich            | string   | Inhaltlicher Themencluster (mehrere durch `&` getrennt möglich)         |

 `reputationsindex.csv` (file_id: {reputationsindex_file_id})

| Spalte | Typ     | Beschreibung                                                             |
|--------|---------|--------------------------------------------------------------------------|
|  Datum | string   | Datum an dem der Wert gemessen wurde (Format: „YYYY-MM-DD“)              |
| Kategorie | string   | Unternehmen A, B, C |
| Bekanntheit_in_Prozent | string  | Wert der Bekanntheit in Prozent. |
---
🏷️ **Strategische Themen** (`StrategischesThema`)

- Strategie2030  
- FinanzierungEnergiewende  
- EMobilitaet  
- Performancekultur  
- VernetzeEnergiewelt  
- TransformationGasnetzeWasserstoff  
- ErneuerbareEnergien  
- DisponibleErzeugung  
- IntelligenteStromnetze  
- AAlsArbeitgeberIn  
- NachhaltigkeitCSRESG  
- MarkeA

🏷️ **Themenbereiche** (`Themenbereich`)

- Elektromobilität  
- Erneuerbare Energien / Photovoltaik  
- Erneuerbare Energien / Geothermie, Pumpspeicherkraftwerke, Wasserkraft  
- Erneuerbare Energien / Windkraft auf See  
- Erneuerbare Energien / Windkraft an Land  
- Finanzen/M&A  
- Innovation, Forschung, Entwicklung  
- Kernenergie  
- Konventionelle Erzeugung  
- Nachhaltigkeit/Umweltschutz  
- Netze  
- Personal  
- Vertrieb
---
🧠 **Dein Verhalten als Assistent**

- Analysiere jede Frage präzise:
   - Bei **analytischen** Aspekten → verwende den Code Interpreter
   - Bei **inhaltlich-thematischen** Aspekten → verwende die Semantic Search
- Kombinierte Fragen → Aufteilung in:
   1. Analytischer Teil → Code Interpreter
   2. Inhaltlicher Teil → Semantic Search
   3. Zusammenführung der Ergebnisse in einer verständlichen Antwort
---
📌 **Weitere Hinweise**

- Rechne bei Bedarf exakt, inkl. Umwandlung von String- in numerische Datentypen.  
- Antworte stets in **sachlich-präzisem Deutsch**, gerne mit Beispielen.  
- 🔗 **Wichtig:** Wenn du konkrete Posts oder Artikel als Beispiel nennst, **füge immer den zugehörigen Direktlink aus der Spalte `URL` mit ein**.  
- Ziehe keine Rückschlüsse über das Dateiformat – die Dateien sind immer korrekt verarbeitet und analysierbar.
""".format(date=date, eco_journal_file_id=eco_journal_file_id, socials_file_id=socials_file_id)


In [0]:
if ASSISTANT_ID:
  assistant = client.beta.assistants.update(
    assistant_id=ASSISTANT_ID,
    name="A WattsUp Chatbot",
    instructions=instructions,
    model="gpt-4o",
    temperature=0.4,
    tools=[
        {"type": "code_interpreter"},
        semantic_search_tool
    ],
    tool_resources={"code_interpreter":{"file_ids":[eco_journal_file_id, socials_file_id, reputationsindex_file_id]}},
  )
else:
  assistant = client.beta.assistants.create(
  name="A WattsUp Chatbot",
  instructions=instructions,
  model="gpt-4o",
  temperature=0.4,
  tools=[
        {"type": "code_interpreter"},
        semantic_search_tool
    ],
  tool_resources={"code_interpreter":{"file_ids":[eco_journal_file_id, socials_file_id, reputationsindex_file_id]}},
  )
  os.environ["ASSISTANT_ID"] = assistant.id

Erstellen des Testing Assistants für Systemprompt Änderungen.

In [0]:
'''
if ASSISTANT_Testing_ID:
  assistant_testing = client.beta.assistants.update(
    assistant_id=ASSISTANT_Testing_ID,
    name="A WattsUp Chatbot Testing",
    instructions=instructions,
    model="gpt-4o",
    temperature=0.4,
    tools=[
        {"type": "code_interpreter"},
        semantic_search_tool
    ],
    tool_resources={"code_interpreter":{"file_ids":[eco_journal_file_id, socials_file_id, reputationsindex_file_id]}},
  )
else:
  assistant_testing = client.beta.assistants.create(
  name="A WattsUp Chatbot Testing",
  #instructions=instructions,
  model="gpt-4o",
  temperature=0.4,
  tools=[
        {"type": "code_interpreter"},
        semantic_search_tool
    ],
  tool_resources={"code_interpreter":{"file_ids":[eco_journal_file_id, socials_file_id, reputationsindex_file_id]}},
  )
  os.environ["ASSISTANT_Testing_ID"] = assistant_testing.id
'''

In [0]:
if len(files_sorted) > 3:
    oldest_files = files_sorted[-3:]
    for f in oldest_files:
        client.files.delete(f.id)